In [11]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
import random
import torch
from torch.utils.data import Dataset, DataLoader
from pymongo import MongoClient
from functools import reduce
from collections import defaultdict
import proteusAI as pai
import os
from typing import Union
from pathlib import Path

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
import random

class CustomSampler(Sampler):
    """
    A custom sampler that can shuffle or sequentially sample the indices.
    """
    def __init__(self, data_source, shuffle=True):
        self.data_source = data_source
        self.shuffle = shuffle
        self.indices = list(range(len(data_source)))
        if self.shuffle:
            random.shuffle(self.indices)
    
    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.indices)
        return iter(self.indices)

    def __len__(self):
        return len(self.data_source)

class CustomBatchSampler(torch.utils.data.Sampler):
    """
    Custom BatchSampler that groups samples into batches of fixed size.
    """
    def __init__(self, sampler, batch_size):
        """
        Args:
            sampler: A custom or standard Sampler object.
            batch_size: Number of samples per batch.
        """
        self.sampler = sampler
        self.batch_size = batch_size

    def __iter__(self):
        batch = []
        for idx in self.sampler:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yield batch
                batch = []
        
        # Handle the case where the last batch is smaller than batch_size
        if len(batch) > 0:
            yield batch

    def __len__(self):
        return (len(self.sampler) + self.batch_size - 1) // self.batch_size

# Example Dataset
class ExampleDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Example data
data = [("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
        ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
        ("protein3", "K A <mask> I S Q"),
        ("protein4", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG")]

# Create Dataset
dataset = ExampleDataset(data)

# Use the CustomSampler and BatchSampler
custom_sampler = CustomSampler(dataset, shuffle=True)  # Shuffle the data
batch_sampler = CustomBatchSampler(custom_sampler, batch_size=2)

# Create DataLoader with the custom BatchSampler
dataloader = DataLoader(dataset, batch_sampler=batch_sampler)

# Iterate through the DataLoader
for batch in dataloader:
    print(batch)


[('protein2', 'protein1'), ('KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE', 'MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG')]
[('protein3', 'protein4'), ('K A <mask> I S Q', 'MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG')]


In [8]:
def get_taxon_sequence_data(collection):

    # Define the query filter to exclude documents where sequence.length > 1024
    query = {
        "sequence.length": {"$lte": 1024}
    }
    projection = {
    "primaryAccession": 1,    
    "organism.taxonId": 1,   # Include taxonId from the organism field
    "sequence.value": 1,      # Include value from the sequence field
    "sequence.length": 1      # Include length from the sequence field
    }
    documents =  collection.find(query, projection)

    minimal_documents = [] # Initialize new empty dictionary

    for doc in documents:
    # Create a new dictionary with only the desired properties
        new_obj = {
            "primaryAccession": doc.get("primaryAccession",{}),
            "taxonId": doc.get("organism", {}).get("taxonId"),
            "value": doc.get("sequence", {}).get("value"),
            "length": doc.get("sequence", {}).get("length")
        }
        minimal_documents.append(new_obj)


    return minimal_documents
    

In [9]:
client, db, collection = connect_db()
data = get_taxon_sequence_data(collection)
print(len(data))


NameError: name 'connect_db' is not defined

In [65]:
class ProteinDataset(Dataset):

    def __init__(self, data):

        self.data = data

    def __len__(self):

        return len(self.data)
    
    def __getitem__(self, index):

        item = self.data[index]

        # Extract relevant fields
        sequence = item['value']  # The protein sequence
        length = item['length']   # Length of the sequence
        taxon_id = item['taxonId']  # Taxon ID
        primary_accession = item['primaryAccession']  # Primary accession

        return {
            'sequence': sequence,  # Return sequence as a string (you could modify this later)
            'length': length,
            'taxon_id': taxon_id,
            'primary_accession': primary_accession
        }
    

In [66]:
dataset = ProteinDataset(data)


In [67]:
class CustomSampler(Sampler):

    def __init__(self, data_source : Dataset, shuffle = True):

        self.data_source = data_source
        self.indices = list(range(len(data_source)))
        self.shuffle = shuffle

    def __iter__(self):
        
        if self.shuffle:
                random.shuffle(self.indices)  # Shuffle the indices if needed
            
        # Return the indices one by one (you could also apply custom logic here)
        for idx in self.indices:
            yield idx
            
    def __len__(self):
        """
        Return the number of items in the dataset.
        """
        return len(self.data_source)



In [68]:
sampler = CustomSampler(dataset, shuffle=True)
dataloader = DataLoader(dataset, batch_size=5, sampler=sampler)

# Iterate over the DataLoader
for batch in dataloader:
    print(batch)  

{'sequence': ['MSSTAGQVINCKAAVAWSAKAPLSIETIQVAPPKAHEVRVKILYTAVCHTDAYTLDGHDPEGLFPVVLGHEGSGIVESVGEGVTGFAPGDHVVPLYVPQCKECEYCKNPKTNLCQKIRISQGNGFMPDGSSRFTCNGKQLFHFMGCSTFSEYTVVADISLCKVNPEAPLEKVSLLGCGISTGYGAVLNTCKVEEGSTVAVWGLGAVGLAVIMGAKAAGAKKIVGIDLIESKFESAKFFGATECINPKSVELPEGKSFQAWLVEQFDGGFDYTFECIGNVHTMRQALEAAHKGWGVSCIIGVAGAGQEIATRPFQLVTGRTWKGTAFGGWKSVESVPRLVDDYMNKKLLIDEFITHRWNIDDINTAFDVLHKGESLRSVLAFEKI', 'MAMRQAAKATIRACSSSSSSGYFARRQFNASSGDSKKIVGVFYKANEYATKNPNFLGCVENALGIRDWLESQGHQYIVTDDKEGPDCELEKHIPDLHVLISTPFHPAYVTAERIKKAKNLKLLLTAGIGSDHIDLQAAAAAGLTVAEVTGSNVVSVAEDELMRILILMRNFVPGYNQVVKGEWNVAGIAYRAYDLEGKTIGTVGAGRIGKLLLQRLKPFGCNLLYHDRLQMAPELEKETGAKFVEDLNEMLPKCDVIVINMPLTEKTRGMFNKELIGKLKKGVLIVNNARGAIMERQAVVDAVESGHIGGYSGDVWDPQPAPKDHPWRYMPNQAMTPHTSGTTIDAQLRYAAGTKDMLERYFKGEDFPTENYIVKDGELAPQYR', 'MSGNRGVVYLGSGKVEVQKIDYPKMQDPRGKKIEHGVILKVVSTNICGSDQHMVRGRTTAQVGLVLGHEITGEVIEKGRDVENLQIGDLVSVPFNVACGRCRSCKEMHTGVCLTVNPARAGGAYGYVDMGDWTGGQAEYLLVPYADFNLLKLPDRDKAMEKIRDLTCLSDILPTGYHGAVTAGVGPGSTVYVAGAGPVGLAAAASARLL

In [34]:
class TaxonIdSampler(Sampler):
    def __init__(self, dataset : Dataset, batch_size, length_bin_size=5, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.length_bin_size = length_bin_size
        self.shuffle = shuffle

        # Group sample indices by taxonId
        self.taxon_length_bins = defaultdict(list)

        for idx, sample in enumerate(dataset):
            taxon_id = sample['taxon_id']
            
            sequence_length = sample['length']
            length_bin = (sequence_length // length_bin_size) *length_bin_size # integer division to know in which bucket the sequence is

            self.taxon_length_bins[taxon_id][length_bin].append(idx) # create dictionary with both separations taxon id and lengths
       
        '''
        structure of self.taxon_length_bins:

                    {
                taxon_id_1: {
                    length_bin_1: [sample_idx_1, sample_idx_2, ...],
                    length_bin_2: [sample_idx_3, sample_idx_4, ...],
                    ...
                },
                taxon_id_2: {
                    length_bin_3: [sample_idx_5, sample_idx_6, ...],
                    ...
                },
                ...

        ''' 

        # Prepare batches based on taxon groups
        self.batches = []

        for taxon, length_bins in self.taxon_length_bins.items():
            for length_bin, indices in length_bin.items():
                if self.shuffle:
                    random.shuffle(indices) 
                for i in range(0, len(indices), batch_size):
                    self.batches.append(indices[i:i + batch_size])
         
         # Shuffle the batches if needed
        if self.shuffle:
            random.shuffle(self.batches)



        # for indices in self.taxon_to_indices.values():
        #     if self.shuffle:
        #         random.shuffle(indices)

        #     for i in range(0,len(indices), batch_size):

        #         self.batches.append(indices[i:i + batch_size])

        #     if self.shuffle:
        #         random.shuffle(self.batches)
    
    def __iter__(self):

        for batch in self.batches:
            yield batch

    def __len__(self):
        return len(self.batches)

In [55]:
def dict_collate_fn(batch):
    # Return the batch as-is (a list of dictionaries)
    return batch

In [71]:
# Initialize the TaxonBatchSampler with your dataset
batch_size = 3
sampler = TaxonIdSampler(dataset, batch_size=batch_size, shuffle=True)

dataloader = DataLoader(dataset, batch_sampler=sampler, collate_fn=dict_collate_fn)

# Iterate over the DataLoader
for batch in dataloader:
    print(batch)  # Each batch will contain samples with the same taxonId

[{'sequence': 'MANQVIKCKAAVAWEAGKPLSIEEVEVAPPKAHEVRIKIIATAVCHTDAYTLSGADPEGNYPVILGHEGAGIVESVGEGVTKLKAGDTVIPLYIPQCGECKFCLNPKTNLCQKIRVTQGKGLMPDGTSRFTCKGKTILHYMGTSTFSEYTVVADISVAKIDPLAPLDKVCLLGCGISTGYGAALNAAKVEPGSTCAVFGLGGVGLAVIMGCKMAGAARIIGVDINKDKFARAKEFGASECINPQDFSKPIQEVLIEMTDGGVDYSFECIGNVKVMRAALEACHKGWGISVVVGVAASGEEIATRPFQLVTGRTWKGTAFGGWKSVESVPKLVSEYMSKKIKVDEFVTHSLPFDQINEAFDLMHAGKSIRTVVKL', 'length': 374, 'taxon_id': 9913, 'primary_accession': 'Q3ZC42'}, {'sequence': 'MANQVIKCKAAVAWEAGKPLSIEEVEVAPPKAHEVRIKIIATAVCHTDAYTLSGADPEGNYPVILGHEGAGIVESVGEGVTKLKAGDTVIPLYIPQCGECKFCLNPKTNLCQKIRVTQGKGLMPDGTSRFTCKGKTILHYMGTSTFSEYTVVADISVAKIDPLAPLDKVCLLGCGISTGYGAALNAAKVEPGSTCAVFGLGGVGLAVIMGCKMAGAARIIGVDINKDKFARAKEFGASECINPQDFSKPIQEVLIEMTDGGVDYSFECIGNVKVMRAALEACHKGWGISVVVGVAASGEEIATRPFQLVTGRTWKGTAFGGWKSVESVPKLVSEYMSKKIKVDEFVTHSLPFDQINEAFDLMHAGKSIRTVVKL', 'length': 374, 'taxon_id': 9913, 'primary_accession': 'Q3ZC42'}]
[{'sequence': 'MKSRAAVAFAPGKPLEIVEIDVAPPKKGEVLIKVTHTGVCHTDAFTLSGDDPEGVFPVVLGHEGAGVVVEVG

In [77]:
import esm

In [96]:
def esm_compute(seqs: list, names: list=None, model: Union[str, torch.nn.Module]="esm1v", rep_layer: int=33, device=None):
    """
    Compute the of esm_tools models for a list of sequences.
 
    Args:
        seqs (list): protein sequences either as str or biotite.sequence.ProteinSequence.
        names (list, default None): list of names/labels for protein sequences.
            If None sequences will be named seq1, seq2, ...
        model (str, torch.nn.Module): choose either esm2, esm1v or a pretrained model object.
        rep_layer (int): choose representation layer. Default 33.
        device (str): Choose hardware for computation. Default 'None' for autoselection
                          other options are 'cpu' and 'cuda'.
 
    Returns: representations (list) of sequence representation, batch lens and batch labels
 
    Example:
        seqs = ["AGAVCTGAKLI", "AGHRFLIKLKI"]
        results, batch_lens, batch_labels = esm_compute(seqs)
    """
    # detect device
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    else:
        device = torch.device(device)
 
    # on M1 if mps available
    #if device == torch.device(type='cpu'):
    #    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
 
    # load model
    if isinstance(model, str):
        if model == "esm2":
            model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
        elif model == "esm1v":
            model, alphabet = esm.pretrained.esm1v_t33_650M_UR90S()
        else:
            raise ValueError(f"{model} is not a valid model")
    elif isinstance(model, torch.nn.Module):
        alphabet = torch.load(os.path.join(Path(__file__).parent, "alphabet.pt"))
    else:
        raise TypeError("Model should be either a string or a torch.nn.Module object")
 
 
    batch_converter = alphabet.get_batch_converter()
    model.eval()
    model.to(device)
 
    if names == None:
        names = names = [f'seq{i}' for i in range(len(seqs))]
 
    data = list(zip(names, seqs))
 
    # check datatype of sequences - str or biotite
    if all(isinstance(x[0], str) and isinstance(x[1], str) for x in data):
        pass  # all elements are strings
    else:
        data = [(x[0], str(x[1])) for x in data]
 
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
 
    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens.to(device), repr_layers=[rep_layer], return_contacts=True)
 
    return results, batch_lens, batch_labels, alphabet

In [100]:
def get_seq_rep(results, batch_lens):
    """
    Get sequence representations from esm_compute
    """
    token_representations = results["representations"][33]
 
    # Generate per-sequence representations via averaging
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1: tokens_len - 1].mean(0))
 
    return sequence_representations
 
 
def get_logits(results):
    """
    Get logits from esm_compute
    """
    logits = results["logits"]
    return logits

In [98]:
seqs = ["AGAVCTGAKLI", "AGHRFLIKLKI"]
results, batch_lens, batch_labels, alphabet = esm_compute(seqs)

/Users/Rogall/miniforge3/envs/ml/lib/python3.12/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


### First logits computation

In [103]:
LOGITS = []
REPRESENTATIOS = []
for batch in dataloader:
    try:
        sequences = [item['sequence'] for item in batch]
        names = [item['primary_accession'] for item in batch]

        results, batch_lens, batch_labels, alphabet = esm_compute(seqs = sequences, names = names )
        
        sequence_representations = get_seq_rep(results, batch_lens)

        for i, x in enumerate(sequence_representations):
            torch.save(x, f"reps/{names[i]}.pt")

    except:
        pass
    # Print or process the data
    # print("Sequences:", seqs)
    # print("Names:", names)

/Users/Rogall/miniforge3/envs/ml/lib/python3.12/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


KeyboardInterrupt: 

In [105]:
tensors = [torch.load('reps/'+f) for f in os.listdir('reps') if f.endswith('.pt')]



/var/folders/mf/wh6pc4zs5bdf1rhq4yhjvvf00000gp/T/ipykernel_59713/1399910260.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensors = [torch.load('reps/'+f) for f in os.

### Convert xml to jason for importing to mongoDB

In [12]:
import xmltodict
import json